In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, f1_score

# 自定义数据集类（无数据增强）
class SimpleDataset(Dataset):
    def __init__(self, data, labels_defect, labels_severity):
        self.data = data
        self.labels_defect = labels_defect
        self.labels_severity = labels_severity

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label_defect = self.labels_defect[idx]
        label_severity = self.labels_severity[idx]
        
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(label_defect, dtype=torch.long), torch.tensor(label_severity, dtype=torch.long)

# 1. 加载数据
dataX = np.loadtxt('dataset1/dataX.txt')  # 每组数据50个点
dataY = np.loadtxt('dataset1/dataY.txt')
dataZ = np.loadtxt('dataset1/dataZ.txt')
dataLabels = np.loadtxt('dataset1/dataLabel.txt')  # 假设每一行有两列

# 将标签分为两个部分：defect 标签和 severity 标签
labels_defect = dataLabels[:, 0].astype(int)  # 道路缺陷标签
labels_severity = dataLabels[:, 1].astype(int)  # 严重程度标签

# 2. 数据预处理：合并 x, y, z 方向的数据
data = np.column_stack((dataX, dataY, dataZ))  # [样本数, 150]

# 3. 划分数据集
X_train, X_test, y_train_defect, y_test_defect, y_train_severity, y_test_severity = train_test_split(
    data, labels_defect, labels_severity, test_size=0.2, stratify=labels_defect, random_state=42
)

# 创建数据集和数据加载器
train_dataset = SimpleDataset(X_train, y_train_defect, y_train_severity)
test_dataset = SimpleDataset(X_test, y_test_defect, y_test_severity)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义CNN模型
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(128 * 6, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)

        self.fc_defect = nn.Linear(64, 3)  # 道路缺陷分类 (3 类)
        self.fc_severity = nn.Linear(64, 4)  # 严重程度分类 (4 类)

    def forward(self, x):
        x = x.view(-1, 50, 3).permute(0, 2, 1)  # (batch_size, 50, 3) -> (batch_size, 3, 50)

        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))

        x = x.view(-1, 128 * 6)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))

        defect_output = self.fc_defect(x)
        severity_output = self.fc_severity(x)

        return defect_output, severity_output

# 创建模型实例
model = CNN_Model()

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            optimizer.zero_grad()
            defect_preds, severity_preds = model(inputs)
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

# 测试模型
def test_model(model, test_loader):
    model.eval()
    all_defect_labels = []
    all_defect_preds = []
    all_severity_labels = []
    all_severity_preds = []
    
    with torch.no_grad():
        for inputs, defect_labels, severity_labels in test_loader:
            defect_preds, severity_preds = model(inputs)
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
    precision_defect = precision_score(all_defect_labels, all_defect_preds, average='weighted', zero_division=0)
    f1_defect = f1_score(all_defect_labels, all_defect_preds, average='weighted', zero_division=0)

    accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)
    precision_severity = precision_score(all_severity_labels, all_severity_preds, average='weighted', zero_division=0)
    f1_severity = f1_score(all_severity_labels, all_severity_preds, average='weighted', zero_division=0)

    print(f'道路缺陷分类准确率: {accuracy_defect * 100:.2f}%')
    print(f'严重程度分类准确率: {accuracy_severity * 100:.2f}%')
    print(f'道路缺陷分类精确度: {precision_defect:.2f}')
    print(f'严重程度分类精确度: {precision_severity:.2f}')
    print(f'道路缺陷分类F1值: {f1_defect:.2f}')
    print(f'严重程度分类F1值: {f1_severity:.2f}')

# 开始训练和测试
train_model(model, train_loader, criterion, optimizer, num_epochs=20)
test_model(model, test_loader)


Epoch [1/20], Loss: 1.7838110476732254
Epoch [2/20], Loss: 1.2819219455122948
Epoch [3/20], Loss: 1.1044483073055744
Epoch [4/20], Loss: 0.9259337037801743
Epoch [5/20], Loss: 0.804098054766655
Epoch [6/20], Loss: 0.696673920378089
Epoch [7/20], Loss: 0.4786099847406149
Epoch [8/20], Loss: 0.42043686378747225
Epoch [9/20], Loss: 0.3358104443177581
Epoch [10/20], Loss: 0.33418988808989525
Epoch [11/20], Loss: 0.28850814048200846
Epoch [12/20], Loss: 0.2384971035644412
Epoch [13/20], Loss: 0.21678446140140295
Epoch [14/20], Loss: 0.22917607799172401
Epoch [15/20], Loss: 0.18854827992618084
Epoch [16/20], Loss: 0.17926976922899485
Epoch [17/20], Loss: 0.18768088752403855
Epoch [18/20], Loss: 0.1347769466228783
Epoch [19/20], Loss: 0.13735283841378987
Epoch [20/20], Loss: 0.10460217436775565
道路缺陷分类准确率: 80.80%
严重程度分类准确率: 71.60%
道路缺陷分类精确度: 0.81
严重程度分类精确度: 0.74
道路缺陷分类F1值: 0.81
严重程度分类F1值: 0.72
